# LLMInteractor Examples

This notebook demonstrates example usage of LLMInteractor with and without code execution.

In [1]:
import os
from llm_interact import LLMConfig, LLMInteractor
from logger import configure_global_logger
import logging
configure_global_logger(logging.INFO, "./examples.log")

# Create checkpoints directory if it doesn't exist
os.makedirs("./checkpoints", exist_ok=True)

%reload_ext autoreload
%autoreload 2


## Example 1: LLMInteractor without code execution

This example uses litellm to call API directly without executing code.

In [2]:
def example_without_code_execution():
    """
    Example of using LLMInteractor without code execution.
    Uses litellm to call API directly.
    """
    print("=== Example: LLMInteractor without code execution ===")

    # Load configuration from TOML file
    config = LLMConfig.from_toml("./llm_config.toml")

    # Make sure code execution is disabled for this example
    config.run_code = False
    config.checkpoint_path = "./checkpoints/no_code_example.json"

    # Initialize the interactor
    interactor = LLMInteractor(config)

    # Send messages to the LLM
    messages = [
        "Hello! Can you briefly explain what quantum computing is?",
        "Can you give me a simple example of a quantum algorithm?",
        "How does that differ from classical computing?",
    ]

    responses = interactor.send_all(messages)

    print("responses are actually the same as interactor.messages")
    print("responses == interactor.messages: ", responses == interactor.messages)
    # Print the conversation
    print("\nConversation history:")
    for i, msg in enumerate(interactor.messages):
        role = msg["role"]
        content = msg["content"]
        print(f"\n[{role.upper()}]:\n{content[:100]}...")  # Show first 100 chars

    # Save checkpoint (this happens automatically during send_all, but can be called manually)
    interactor.store_checkpoint()
    print(f"\nCheckpoint saved to {config.checkpoint_path}")

In [3]:
# Run the example without code execution
try:
    example_without_code_execution()
except Exception as e:
    print(f"Error in non-code example: {str(e)}")

=== Example: LLMInteractor without code execution ===
2025-04-27 20:10:16 - DataSciBench - INFO - Loaded configuration from ./llm_config.toml
2025-04-27 20:10:16 - DataSciBench - INFO - Initialized LLMInteractor with model: gemini/gemini-2.0-flash-lite, temperature: 0.4
2025-04-27 20:10:16 - DataSciBench - INFO - Sending 3 messages to LLM
2025-04-27 20:10:16 - DataSciBench - INFO - Checkpoint saved to ./checkpoints/no_code_example.json
2025-04-27 20:10:16 - DataSciBench - INFO - Calling LLM API with a message (total length: ~57 chars)
2025-04-27 20:10:17 - DataSciBench - INFO - LLM API call successful on attempt 1
2025-04-27 20:10:17 - DataSciBench - INFO - Checkpoint saved to ./checkpoints/no_code_example.json
2025-04-27 20:10:17 - DataSciBench - INFO - Calling LLM API with a message (total length: ~56 chars)
2025-04-27 20:10:20 - DataSciBench - INFO - LLM API call successful on attempt 1
2025-04-27 20:10:20 - DataSciBench - INFO - Checkpoint saved to ./checkpoints/no_code_example.jso

## Example 2: LLMInteractor with code execution

This example uses open-interpreter to execute code.

In [10]:
def example_with_code_execution():
    """
    Example of using LLMInteractor with code execution.
    Uses open-interpreter to execute code.
    """
    print("\n=== Example: LLMInteractor with code execution ===")

    # Load configuration from TOML file
    config = LLMConfig.from_toml("./llm_config.toml")

    # Enable code execution for this example
    config.run_code = True
    config.checkpoint_path = "./checkpoints/code_execution_example.json"

    # Initialize with interpreter config
    interpreter_config_path = "./interpreter_config.toml"
    interactor = LLMInteractor(config, interpreter_config_path=interpreter_config_path)

    # Send messages with coding tasks
    messages = [
        "Generate a python function to calculate the Fibonacci sequence up to n terms. Then run it with n=10. Your code will be executed and I'll give you the print result on the console.",
        "What is the time complexity of your implementation?",
    ]

    responses = interactor.send_all(messages)

    print("responses are actually the same as interactor.messages")
    print("responses == interactor.messages: ", responses == interactor.messages)

    # Print summary of conversation
    print("\nCode execution completed. Check the full conversation in the checkpoint file.")
    print(f"Checkpoint saved to {config.checkpoint_path}")

In [11]:
# Run the example with code execution
try:
    example_with_code_execution()
except Exception as e:
    print(f"Error in code execution example: {str(e)}")


=== Example: LLMInteractor with code execution ===
2025-04-27 21:44:10 - DataSciBench - INFO - Loaded configuration from ./llm_config.toml
2025-04-27 21:44:10 - DataSciBench - INFO - Initialized LLMInteractor with model: gemini/gemini-2.0-flash-lite, temperature: 0.4
2025-04-27 21:44:10 - DataSciBench - INFO - Sending 1 messages to LLM
2025-04-27 21:44:10 - DataSciBench - INFO - Checkpoint saved to ./checkpoints/test.json
2025-04-27 21:44:10 - DataSciBench - INFO - Calling LLM API with a message (total length: ~40 chars)
2025-04-27 21:44:11 - DataSciBench - INFO - LLM API call successful on attempt 1
2025-04-27 21:44:11 - DataSciBench - INFO - Checkpoint saved to ./checkpoints/test.json
responses are actually the same as interactor.messages
responses == interactor.messages:  True

Code execution completed. Check the full conversation in the checkpoint file.
Checkpoint saved to ./checkpoints/test.json


## Example 3: Loading from checkpoint

This example loads a conversation from a checkpoint file and continues it.

In [6]:
def example_load_from_checkpoint():
    """
    Example of loading conversation from a checkpoint file and continuing it.
    """
    print("\n=== Example: Loading from checkpoint ===")

    # create interactor wtih checkpoint
    checkpoints_path = "./checkpoints/no_code_example.json"
    interactor = LLMInteractor.load_from_checkpoint(checkpoints_path)
    print(f"Loaded checkpoint from {checkpoints_path}")
    print("Current conversation history:")
    for i, msg in enumerate(interactor.messages):
        role = msg["role"]
        content = msg["content"]
        print(f"\n[{role.upper()}]:\n{content[:100]}...")  # Show first 100 chars

    # change the checkpoint path
    interactor.config.checkpoint_path = "./checkpoints/continue_example.json"
    # Add a new message to the conversation
    new_messages = ["Can you summarize the conversation so far?", "Have a great day!"]
    interactor.send_all(new_messages)
    print("New messages added to the conversation.")
    print("Updated conversation history:")
    for i, msg in enumerate(interactor.messages):
        role = msg["role"]
        content = msg["content"]
        print(f"\n[{role.upper()}]:\n{content[:100]}...")  # Show first 100 chars
    # Save the updated conversation to a new checkpoint (this happens automatically during send_all, but can be called manually)
    interactor.store_checkpoint()
    print(f"\nCheckpoint saved to {interactor.config.checkpoint_path}")

In [7]:
# Run the example loading from checkpoint
try:
    example_load_from_checkpoint()
except Exception as e:
    print(f"Error in checkpoint loading example: {str(e)}")


=== Example: Loading from checkpoint ===
2025-04-27 20:34:44 - DataSciBench - INFO - Initialized LLMInteractor with model: gemini/gemini-2.0-flash-lite, temperature: 0.4
2025-04-27 20:34:44 - DataSciBench - INFO - Loaded checkpoint from ./checkpoints/no_code_example.json
Loaded checkpoint from ./checkpoints/no_code_example.json
Current conversation history:

[USER]:
Hello! Can you briefly explain what quantum computing is?...

[ASSISTANT]:
Quantum computing is a new type of computing that uses the principles of quantum mechanics (the phys...

[USER]:
Can you give me a simple example of a quantum algorithm?...

[ASSISTANT]:
A very simple example of a quantum algorithm is **Deutsch's Algorithm**. It's more of a conceptual d...

[USER]:
How does that differ from classical computing?...

[ASSISTANT]:
The key differences between quantum computing and classical computing lie in how they store and proc...
2025-04-27 20:34:44 - DataSciBench - INFO - Sending 2 messages to LLM
2025-04-27 20:34: